# Init

In [1]:
import os
os.chdir('..')

/Users/wliao0504/code/apt-plus


/Users/wliao0504/code/apt-plus/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Import

In [282]:
import duckdb
import pandas as pd
from openai import OpenAI
import json
import re
from importlib import reload
import csv

In [283]:
import scripts.inspections_utils as iu
reload(iu)
from scripts.inspections_utils import (
    filter_by_violation_code,
    generate_summary_of_trivial_violations,
    filter_df_by_address,
    generate_prompt_from_address,
    generate_concat_report_for_one_occasion,
    generate_concat_report_for_all_occasions,
    remove_trivial_violations_by_code,
    TRIVIAL_VIOLATION_CODES,
    BASE_PROMPT,
    ADDRESSES_TO_CHECK,
    clean_json_string,
    llm_summarize_for_one_address,
    filter_for_hyde_park
)

## Load data

In [227]:
df_raw = pd.read_csv("data/Building_Violations_20250421.csv")

In [228]:
df_all = iu.filter_and_recast_columns(df_raw)

There are 1954065 rows, with 171017 unique addresses


In [229]:
df_recent = iu.filter_for_recent(df_all)

There are 264488 rows, with 42831 unique addresses


In [230]:
df_periodic = iu.filter_by_inspection_categories(df_recent, ["PERIODIC"])

There are 51955 rows, with 8917 unique addresses


### recent (since 2020) periodic and complaint

In [231]:
df_recent_subset = iu.filter_by_inspection_categories(df_recent, ["PERIODIC", "COMPLAINT"])

There are 238661 rows, with 37027 unique addresses


### HP

In [232]:
df_hp = df_raw \
    .pipe(iu.filter_for_hyde_park) \
    .pipe(iu.filter_and_recast_columns) \
    .pipe(iu.filter_for_recent) \
    .pipe(iu.filter_by_inspection_categories, ["PERIODIC", "COMPLAINT"])

There are 37997 rows, with 2480 unique addresses
There are 37997 rows, with 2480 unique addresses
There are 6249 rows, with 836 unique addresses
There are 5472 rows, with 706 unique addresses


# EDA

### Inspection Category

In [24]:
df_all.value_counts("INSPECTION CATEGORY")

INSPECTION CATEGORY
COMPLAINT       1377449
PERIODIC         475334
PERMIT            98689
REGISTRATION       2593
Name: count, dtype: int64

### Violation Code

check if violation code -> violation description mapping is one-to-one. turns out one code would only map to one description, but one description might map to multiple codes.

In [189]:
query = """
SELECT 
    "VIOLATION CODE",
FROM df_all
GROUP BY "VIOLATION CODE"
HAVING COUNT(DISTINCT "VIOLATION DESCRIPTION") > 1
"""
duckdb.sql(query).df()

,VIOLATION CODE


In [190]:
query = """
SELECT 
    "VIOLATION DESCRIPTION",
FROM df_all
GROUP BY "VIOLATION DESCRIPTION"
HAVING COUNT(DISTINCT "VIOLATION CODE") > 1
"""
duckdb.sql(query).df()

,VIOLATION DESCRIPTION
0,None
1,EXAMINER'S COMMENTS:
2,PROVIDE
3,DEFECTIVE METER EQUIPMENT
4,MAINT PLATFORM LIFT
...,...
105,DEFECTIVE LIGHTING SERVICE
106,NO PERMIT AWNING SIGN
107,"PROVIDE SOIL-WASTE, VENT PIPES"
108,PROVIDE ACCESS TO PIT


In [191]:
source_df = "df_recent_subset"
query = f"""
WITH violation_counts AS (
  SELECT
    "VIOLATION DESCRIPTION",
    "VIOLATION CODE",
    COUNT(DISTINCT "VIOLATION CODE") OVER (PARTITION BY "VIOLATION DESCRIPTION") AS n_code_per_desc,
    COUNT(*) OVER (PARTITION BY "VIOLATION DESCRIPTION", "VIOLATION CODE") AS n_violations
  FROM {source_df}
),
prioritized AS (
  SELECT *,
         MAX(n_violations) OVER (PARTITION BY "VIOLATION DESCRIPTION") AS max_desc_violations
  FROM violation_counts
),
grouped_ranked AS (
  SELECT *,
         DENSE_RANK() OVER (ORDER BY max_desc_violations DESC) AS group_priority
  FROM prioritized
)
SELECT DISTINCT
  "VIOLATION DESCRIPTION",
  "VIOLATION CODE",
  n_code_per_desc,
  n_violations
FROM grouped_ranked
ORDER BY group_priority, "VIOLATION DESCRIPTION", n_violations DESC
"""
code_to_description = duckdb.sql(query).df()

In [192]:
code_to_description[code_to_description["VIOLATION DESCRIPTION"] == 'ARRANGE PREMISE INSPECTION']

,VIOLATION DESCRIPTION,VIOLATION CODE,n_code_per_desc,n_violations
0,ARRANGE PREMISE INSPECTION,CN190019,2,14727
1,ARRANGE PREMISE INSPECTION,CN193305,2,3


In [193]:
df_all["VIOLATION CODE"].value_counts()

VIOLATION CODE
CN190019       104670
EV1110          68708
CN061014        63171
CN196029        57834
CN070024        50343
                ...  
EV0325              1
NC1495              1
FD CIT 04-C         1
RN1697              1
HSF20               1
Name: count, Length: 1566, dtype: int64

In [194]:
df_all["VIOLATION DESCRIPTION"].value_counts()

VIOLATION DESCRIPTION
ARRANGE PREMISE INSPECTION        104682
MAINTAIN OR REPAIR ELECT ELEVA     68708
REPAIR EXTERIOR WALL               63171
POST OWNER/MANAGERS NAME/#         57834
REPAIR PORCH SYSTEM                50343
                                   ...  
REP/REPL DEF DR CLOSE BUTTON P         1
VENT CAR ENCLOSURE PASS                1
REP/REPL CAR/TOP OPER/DEVICE F         1
PRVD UNOBST EGRESS F/VCNTCOMM          1
INSECTS AND VERMIN CONTROL             1
Name: count, Length: 1396, dtype: int64

In [202]:
code_to_description[code_to_description["VIOLATION DESCRIPTION"].str.contains("REINSPECT", na=False)]

,VIOLATION DESCRIPTION,VIOLATION CODE,n_code_per_desc,n_violations
23,ARRANGE FOR REINSPECTION REGAR,CN190029,1,2738


### Inspection Status

check if the inspection id would always get the same inspecton status

In [22]:
query = """
SELECT 
    "INSPECTION ID",
    COUNT(DISTINCT "INSPECTION STATUS") as n_status
FROM df_all
GROUP BY "INSPECTION ID"
ORDER BY n_status DESC 
"""
duckdb.sql(query).df()

BinderException: Binder Error: Referenced column "INSPECTION NUMBER" not found in FROM clause!
Candidate bindings: "INSPECTION ID", "INSPECTION CATEGORY", "INSPECTION STATUS", "VIOLATION INSPECTOR COMMENTS", "VIOLATION ORDINANCE"

In [16]:
df_all[df_all["INSPECTION NUMBER"] == 11219187]

,ID,VIOLATION LAST MODIFIED DATE,VIOLATION DATE,VIOLATION CODE,VIOLATION STATUS,VIOLATION STATUS DATE,VIOLATION DESCRIPTION,VIOLATION LOCATION,VIOLATION INSPECTOR COMMENTS,VIOLATION ORDINANCE,...,ADDRESS,STREET NUMBER,STREET DIRECTION,STREET NAME,STREET TYPE,PROPERTY GROUP,SSA,LATITUDE,LONGITUDE,LOCATION
921720,4693785,11/09/2016,03/27/2014,RF302041,COMPLIED,11/09/2016,PRV HIGH SIDE,BUILDING,FOR TWO REZNOR 8 TON RTU'S WITH DUAL RECEIVERS...,Install safety valve in high pressure side of ...,...,6033 S COTTAGE GROVE AVE,6033,S,COTTAGE GROVE,AVE,23366,NaN,41.784883,-87.605882,"(41.78488270328645, -87.60588249894568)"


test if unique inspection ids are spread across multiple days

In [35]:
query = f"""
SELECT 
    "INSPECTION NUMBER", 
    COUNT(DISTINCT strptime("VIOLATION DATE", '%m/%d/%Y')) as n_days,
    COUNT(DISTINCT strftime(strptime("VIOLATION DATE", '%m/%d/%Y'), '%Y-%m')) as n_months
FROM df_all
GROUP BY "INSPECTION NUMBER"
HAVING n_months > 1
"""
duckdb.sql(query).df()

,INSPECTION NUMBER,n_days,n_months
0,10278175,2,2
1,10727949,2,2
2,2290066,2,2
3,2272500,3,2
4,10695717,2,2
...,...,...,...
37725,10374141,2,2
37726,14389163,2,2
37727,13073394,2,2
37728,12395971,2,2


In [27]:
df_all["INSPECTION NUMBER"].nunique()

451381

In [36]:
df = df_all[df_all["INSPECTION NUMBER"] == 10278175]
df.head()

,ID,VIOLATION LAST MODIFIED DATE,VIOLATION DATE,VIOLATION CODE,VIOLATION STATUS,VIOLATION STATUS DATE,VIOLATION DESCRIPTION,VIOLATION LOCATION,VIOLATION INSPECTOR COMMENTS,VIOLATION ORDINANCE,...,ADDRESS,STREET NUMBER,STREET DIRECTION,STREET NAME,STREET TYPE,PROPERTY GROUP,SSA,LATITUDE,LONGITUDE,LOCATION
1039749,4346610,12/17/2012,12/17/2012,CN190019,OPEN,NaN,ARRANGE PREMISE INSPECTION,OTHER : :,INTERIOR OF BUILDING NO RESPONSE. UNABLE TO I...,Arrange for inspection of premises. (13-12-100),...,907 W ARGYLE ST,907,W,ARGYLE,ST,11409,34.0,41.973232,-87.652874,"(41.973232363026156, -87.65287395668437)"
1289559,3650234,12/17/2012,10/28/2010,CN140016,NO ENTRY,NaN,DWELLING CLEAN AND SANITARY,INTERIOR:001 :.,"1ST FLOOR CORRIDORS, CARPET IS STAINED THRU OUT .","Keep premises clean, sanitary, and safe. (13-1...",...,907 W ARGYLE ST,907,W,ARGYLE,ST,11409,34.0,41.973232,-87.652874,"(41.973232363026156, -87.65287395668437)"
1289560,3650235,12/17/2012,10/28/2010,CN197087,NO ENTRY,NaN,CARB MONOX DETECT IN RESID,INTERIOR:001 :.,"LAUNDRY ROOM, MISSING CARBON MONOXIDE DETECTOR.",Install carbon monoxide detector within 40 fee...,...,907 W ARGYLE ST,907,W,ARGYLE,ST,11409,34.0,41.973232,-87.652874,"(41.973232363026156, -87.65287395668437)"


# Generate LLM prompts

### 5514 S BLACKSTONE AVE

In [48]:
prompt_5514 = generate_prompt_from_address("5514 S BLACKSTONE AVE", df_recent_subset, BASE_PROMPT)
print(prompt_5514)


Your goal is to summarize lengthy inspection records for an apartment building into a succinct, 
reader-friendly format that is digestible and informative for a tenant searching for an apartment.
You will be provided a concatenated report of alleged violations, and you should output a json 
object like the following:

{
  "summary": // a one-liner summary of the building's overall condition
  "summarized_issues": [
    {
      "date": "Jan 04, 2025", // following the Mon DD, YYYY format
      "issues": [
        {
          "emoji": "🧊", // use a relevant emoji and if possible a different one for each issue
          "description": "Insufficient heating (60°F in living room and kitchen) and low hot water pressure in Unit a, b, c" //
        },
      ]
    },'
    {
      "date": "Mar 15, 2024",
      "issues": [
        {
          "emoji": "🎨",
          "description": "Graffiti and overflowing trash in rear of building"
        },
      ]
    }
  ]
}

The key is for each issue and i

### 5132 S CORNELL AVE

In [ ]:
prompt_5132 = generate_prompt_from_address("5132 S CORNELL AVE", df_recent_subset, BASE_PROMPT)
print(prompt_5132)


You are a helpful assistant that summarizes long inspection records for an apartment building into a one-liner summary as well as succinct bullet points of issues spotted on each occasion. You will be provided a report that is a concatenation of violation records, and you will output a json object like the following: "

{
  "summary": "This building was recently cited for violations related to <...> and <...> concerns, including <one predominant issue such as 'inadequate heating in at least one unit'> and <one or two other issues>.",
  "summarized_issues": [
    {
      "date": "Jan 04, 2025", // following the Mon DD, YYYY format
      "issues": [
        {
          "emoji": "🧊", // use a relevant emoji and if possible a different one for each issue
          "description": "Insufficient heating (60°F in living room and kitchen) and low hot water pressure in Unit a, b, c" //
        },
      ]
    },'
    {
      "date": "Mar 15, 2024",
      "issues": [
        {
          "emoji": 

# Test performance by LLM APIs

## Utils

In [ ]:
address = "5132 S CORNELL AVE"

## Deepseek

In [ ]:
ds_return = llm_summarize_for_one_address(iu.ADDRESSES_TO_CHECK[0], df = df_recent_subset, model = "deepseek/deepseek-prover-v2:free")
print(ds_return)

There are 66 rows, with 1 unique addresses
There are 66 rows, with 1 unique addresses
There are 66 rows, with 1 unique addresses
```json
{
  "summary": "This building was recently cited for alleged violations around <safety and maintenance> concerns, including <inadequate heating, missing fire safety equipment, and improper electrical installations> and <elevator and plumbing issues, including lack of emergency lighting and missing permits>.",
  "summarized_issues": [
    {
      "date": "May 03, 2024",
      "issues": [
        {
          "emoji": "🚨",
          "description": "Fire safety issues: missing or non-illuminated exit signs, missing smoke detectors, and stairway doors improperly held open (all floors and stairways)."
        },
        {
          "emoji": "🔌",
          "description": "Electrical issues: defective lights and missing emergency lights (hallways and stairways)."
        },
        {
          "emoji": "🚪",
          "description": "General maintenance: unsec

## GPT 4.1

In [169]:
gpt_41_return = llm_summarize_for_one_address(address, df = df_recent_subset, model = "openai/gpt-4.1")
print(gpt_41_return)

There are 12 rows, with 1 unique addresses
There are 12 rows, with 1 unique addresses
There are 12 rows, with 1 unique addresses
```json
{
  "summary": "This building was cited in June 2020 for multiple structural and cleanliness concerns, including unsanitary conditions in shared spaces, plumbing and sewage problems, and various safety hazards such as damaged porch components and overgrown landscaping blocking exits.",

  "summarized_issues": [
    {
      "date": "Jun 15, 2020",
      "issues": [
        {
          "emoji": "🚫",
          "description": "Stagnant sewage water found in basement, creating health risks. (Basement)"
        },
        {
          "emoji": "🚽",
          "description": "Clogged and non-functioning toilet, loose sink in rear bathroom."
        },
        {
          "emoji": "🧹",
          "description": "Rear yard and porch cluttered with trash, debris, and fecal matter resulting in unsanitary conditions. (Rear yard/porch)"
        },
        {
         

## Gemini 2.5

In [ ]:
gemini_25_return = llm_summarize_for_one_address(
    address=ADDRESSES_TO_CHECK[3], df = df_recent_subset, model = iu.GEMINI_25, 
    )
print(gemini_25_return)

```json
{
  "summary": "This building was recently cited for alleged violations around heating and safety concerns, including inadequate heating in at least one unit and missing smoke detector batteries, as well as several fire safety hazards.",
  "summarized_issues": [
    {
      "date": "Jan 22, 2025",
      "issues": [
        {
          "emoji": "🌡️",
          "description": "Insufficient heating (60°F in living room and kitchen) with the heating system on, and the use of a cooking stove as a heating device (Unit A2)."
        },
        {
          "emoji": "🌬️",
          "description": "Drafty windows with gaps and air seepage (Unit A2)."
        },
        {
          "emoji": "🚨",
          "description": "Missing batteries for smoke detector (Unit A2)."
        }
      ]
    },
    {
      "date": "Mar 29, 2024",
      "issues": [
        {
          "emoji": "🚪",
          "description": "Missing or out-of-service stairwell and exit signs, and doors held open with wedges 

# Summarize in batch and ingest to DB

In [261]:
# find all unique addresses to summarize
addresses_to_summarize = df_hp.value_counts("ADDRESS").reset_index()["ADDRESS"]
len(addresses_to_summarize)

706

In [268]:
addresses_to_summarize

0              727 E 60TH ST
1          5220 S HARPER AVE
2          6042 S VERNON AVE
3              500 E 47TH ST
4       5445 S INGLESIDE AVE
               ...          
701       5302 S CORNELL AVE
702    5300 S HYDE PARK BLVD
703        5218 S DREXEL AVE
704       5140 S KENWOOD AVE
705      5335 S WOODLAWN AVE
Name: ADDRESS, Length: 706, dtype: object

In [272]:
# Testing on a few examples
for idx, address in addresses_to_summarize[-2:].items():
    print(idx)
    iu.generate_summary_for_one_address(address, df_hp, iu.GEMINI_25)

704
ADDRESS: 5140 S KENWOOD AVE
SUMMARY: {'summary': 'This building was cited recently for alleged violations around structural integrity concerns, including a rusted iron porch.', 'summarized_issues': [{'date': 'Apr 25, 2023', 'issues': [{'emoji': '🏗️', 'description': 'Rusted posts and steel beams on a 3-story iron porch requiring repair (52 St Side).'}]}]}

----------------------------
705
ADDRESS: 5335 S WOODLAWN AVE
SUMMARY: {'summary': 'This building was cited recently for alleged violations around heating concerns, including insufficient heating in at least one unit.', 'summarized_issues': [{'date': 'Jan 18, 2024', 'issues': [{'emoji': '🌡️', 'description': 'Insufficient heating (61°F) during heating season.'}]}]}

----------------------------


## Write to db directly

In [278]:
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config.settings")
from apt_app.models import InspectionSummary

In [279]:
# Define the CSV file name for logging errors
ERROR_CSV_NAME = "data/error_addresses.csv"

# Function to append an address to the CSV file
def log_error_address(address, error_message):
    with open(ERROR_CSV_NAME, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([address, error_message])

In [287]:
# ref: https://docs.djangoproject.com/en/5.2/ref/models/querysets/#update-or-create
for idx, address in addresses_to_summarize.items():
    print(f"Processing address no. {idx+1}: {address}")
    try:
        summ = iu.generate_summary_for_one_address(address, df_hp, iu.GEMINI_25)
        if summ is None:
            continue
        # Write the result directly to db
        obj, created = await InspectionSummary.objects.aupdate_or_create(
            address=address,
            defaults={"summary": summ},
        )
        if created:
            print(f"Created new inspection summary for {address}")
        else:
            print(f"Updated inspection summary for {address}")
    except Exception as e:
        print(f"Error processing address {address}: {e}")
        log_error_address(address, e)
        continue

Processing address no. 1: 727 E 60TH ST
ADDRESS: 727 E 60TH ST
Updated inspection summary for 727 E 60TH ST
Processing address no. 2: 5220 S HARPER AVE
ADDRESS: 5220 S HARPER AVE
Updated inspection summary for 5220 S HARPER AVE
Processing address no. 3: 6042 S VERNON AVE
ADDRESS: 6042 S VERNON AVE
Updated inspection summary for 6042 S VERNON AVE
Processing address no. 4: 500 E 47TH ST
ADDRESS: 500 E 47TH ST
Updated inspection summary for 500 E 47TH ST
Processing address no. 5: 5445 S INGLESIDE AVE
ADDRESS: 5445 S INGLESIDE AVE
Updated inspection summary for 5445 S INGLESIDE AVE
Processing address no. 6: 6100 S DR MARTIN L KING JR DR
ADDRESS: 6100 S DR MARTIN L KING JR DR
Updated inspection summary for 6100 S DR MARTIN L KING JR DR
Processing address no. 7: 4830 S DREXEL BLVD
ADDRESS: 4830 S DREXEL BLVD
Updated inspection summary for 4830 S DREXEL BLVD
Processing address no. 8: 6124 S DR MARTIN L KING JR DR
ADDRESS: 6124 S DR MARTIN L KING JR DR
Updated inspection summary for 6124 S DR 

## Write to local .json
This is theoretically slow and not recommended.

In [241]:
SUMMARY_JSON_NAME = "inspection_summaries_20250525.json"
ERROR_ADDRESSES_NAME = "error_addresses_20250525.json"

In [179]:
# Load existing data if the file exists
try:
    with open(SUMMARY_JSON_NAME, "r") as file:
        results = json.load(file)
except FileNotFoundError:
    results = {}

# Load existing addresses with issues if the file exists
try:
    with open(ERROR_ADDRESSES_NAME, "r") as file:
        addresses_with_issues = json.load(file)
except FileNotFoundError:
    addresses_with_issues = []

for idx, address in enumerate(addresses_to_summarize):
    print(f"Processing address no. {idx}: {address}")
    try:
        summ = generate_summary_of_trivial_violations(address)
        if summ is None:
            continue
        results[address] = summ
        # Write the updated results back to the file
        with open(SUMMARY_JSON_NAME, "w") as file:
            json.dump(results, file, indent=4)
    except Exception as e:
        print(f"Error processing address {address}: {e}")
        addresses_with_issues.append(address)
        continue

# Write the addresses with issues back to the file
with open(ERROR_ADDRESSES_NAME, "w") as file:
    json.dump(addresses_with_issues, file, indent=4)

Processing address no. 0: 5325 S COTTAGE GROVE AVE
Error parsing JSON: 'NoneType' object is not subscriptable
Processing address no. 1: 6040 S HARPER AVE
Processing address no. 2: 4917 S DREXEL BLVD
Processing address no. 3: 727 E 60TH ST
Processing address no. 4: 6158 S DR MARTIN L KING JR DR
Processing address no. 5: 6126 S GREENWOOD AVE
Processing address no. 6: 6211 S KIMBARK AVE
Processing address no. 7: 4900 S DREXEL BLVD
Processing address no. 8: 6253 S COTTAGE GROVE AVE
Processing address no. 9: 6103 S WOODLAWN AVE
Processing address no. 10: 5496 S HYDE PARK BLVD
Processing address no. 11: 5336 S HYDE PARK BLVD
Processing address no. 12: 5130 S DREXEL AVE
Processing address no. 13: 4751 S VINCENNES AVE
Processing address no. 14: 724 E 50TH PL
Processing address no. 15: 6107 S ST LAWRENCE AVE
Processing address no. 16: 5019 S DREXEL BLVD
Error parsing JSON: Expecting value: line 1 column 1 (char 0)
Processing address no. 17: 6206 S RHODES AVE
Processing address no. 18: 6140 S DR